# Compile to Custom Ring Topology

In this notebook, a .qasm benchmark is compiled to a ring topology and saved to an output file. The size of this ring device is of size n where n is the number of qubits used in the quantum algorithm. This custom device uses the native gate library of the IBM quantum machines.

In [52]:
import re
import numpy as np
import sys
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit.dagcircuit import DAGCircuit
from qiskit import compiler
from qiskit.qasm import Qasm
import networkx as nx

print('qiskit vers.= %s'%qiskit.__version__)

qiskit vers.= 0.9.0


In [53]:
def two_qubit_count(program):
    counter = 0
    if type(program) == QuantumCircuit:
        #ibm circuit datastructure
        p = program.qasm()
        p_split = p.split(';')
        for item in p_split:
            item2= item.split()
            if item2 != []:
                if item2[0].lower() == 'cx':
                    counter = counter +1
    return counter

def compute_depth_ibm(p):
    qubit_dict={}
    p_split = p.split(';')
    for item in p_split:
        item2= item.split()
        if item2 != []:
            if ((item2[0].lower() !='openqasm')and
                (item2[0].lower() !='include')and
                (item2[0].lower() !='qreg')and
                (item2[0].lower() !='creg')):
                qubits = [int(s) for s in re.findall(r'-?\d+\.?\d*', item2[1])]
                for thing in qubits:
                    if thing in qubit_dict:
                        qubit_dict[thing] = qubit_dict[thing] +1
                    else:
                        qubit_dict[thing] =1
                        
                        
    max_key = max(qubit_dict, key=qubit_dict.get)
    return qubit_dict[max_key]

def compile_ibm(num_qubits,topology,program):
    if topology.lower() == 'ring':
        edge_list = []
        qr = [None]*num_qubits
        for i in range(0,num_qubits):
            edge_list.append([i,(i+1)%num_qubits])
            qr[i]=i
        
        topology = nx.from_edgelist(edge_list)



        executable = compiler.transpile(circuits=program, basis_gates="cx,u1,u2,u3",seed_transpiler=1,
                                          coupling_map=edge_list,initial_layout=qr)
        depth = executable.depth()
        volume = executable.size()
        q2_count = two_qubit_count(executable)
        qasm = executable.qasm()
        print(qasm)
        out_str = qasm
        out_str = out_str + ("//DEPTH (returned by QISKIT): %s |VOL.: %s |2Q GATE COUNT: %s \n" %(depth,volume,q2_count))
        out_str = out_str + ("//calculated depth (max gates/qubit): %s\n"%(compute_depth_ibm(qasm)))
        print("DEPTH (returned by QISKIT): %s |VOL.: %s |2Q GATE COUNT: %s " %(depth,volume,q2_count))
        print("calculated depth (max gates/qubit): %s"%(compute_depth_ibm(qasm)))
        print()
        print()
        return out_str

In [54]:
benchmark = 'sample_files/full_adder.qasm'
qasm_in = open(benchmark).read()
topology = 'ring'


p_qasm = QuantumCircuit.from_qasm_file(benchmark)
qubits_used = p_qasm.width()
print("Your compiled qasm program is:")

qasm_out = compile_ibm(qubits_used,topology,p_qasm)

f_qasm = open(benchmark[0:-5]+'_cmpled.qasm','w')
f_qasm.write(qasm_out)
f_qasm.close()


Your compiled qasm program is:


//anaconda3/envs/quantum_env/lib/python3.7/site-packages/qiskit/compiler/transpile.py:244: DeprecationWarning: The parameter basis_gates is now a list of strings. For example, this basis ['u1','u2','u3','cx'] should be used instead of 'u1,u2,u3,cx'. The string format will be removed after 0.9
  basis_gates = _parse_basis_gates(basis_gates, backend, circuits)


OPENQASM 2.0;
include "qelib1.inc";
qreg q[10];
creg c[5];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
u2(0.0,3.14159265358979) q[3];
u2(0.0,3.14159265358979) q[4];
cx q[3],q[4];
u2(0.0,3.14159265358979) q[3];
u2(0.0,3.14159265358979) q[4];
cx q[3],q[4];
u2(0.0,3.14159265358979) q[3];
u2(0.0,3.14159265358979) q[4];
cx q[3],q[4];
cx q[2],q[3];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u1(-0.785398163397448) q[3];
cx q[2],q[3];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u1(0.785398163397448) q[3];
cx q[2],q[3];
u1(0.785398163397448) q[2];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u2(0.0,3.14159265358979) q[1];
u2(0.0,3.14159265358979) q[2];
cx q[1],q[2];
u2(0.0,3.14159265358979) q